In [1]:
# Implement Re-Ranking in RAG 

#1. Setup the env and load the necessary libraries
#Load a PDF document, extract text and split it into chunks
#Generate embeddings for the chunks and store them in FAISS
#Perform a query on the FAISS Index and re-rank the retrieved documents using a cross-encoder model
#Generate a response using OpenAI's GPT model based on the top-ranked documents. {To Do}


In [2]:

import os
from pypdf import PdfReader
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer, CrossEncoder
import openai
from dotenv import load_dotenv



C:\Users\suman\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [3]:
# Document Loading and Processing

reader=PdfReader(r"microsoft-annual-report.pdf")
pdf_texts=[p.extract_text().strip() for p in reader.pages]

#filter out any empty strings
pdf_texts=[text for text in pdf_texts if text]

#Split the document into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

character_splitter=RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", "."," ", ""], chunk_size=1000, chunk_overlap=0
)
character_split_texts=character_splitter.split_text("\n\n".join(pdf_texts))
print(f"Number of text chunks:{len(character_split_texts)}")

Number of text chunks:409


In [4]:
#Embedding Generation

embedding_model=SentenceTransformer('all-MiniLM-L6-v2')
embeddings=embedding_model.encode(character_split_texts)

print(f"Embedding Shape:{embeddings.shape}")

C:\Users\suman\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Embedding Shape:(409, 384)


In [5]:
# Retrieval of documents using FAISS

#Create FAISS index
dimension=embeddings.shape[1]
index=faiss.IndexFlatL2(dimension)
index.add(embeddings)

#Define the query and generate it's embeddings
query_text=input("Enter your query")
query_embedding=embedding_model.encode([query_text])

#search for similar embeddings
k=10  # Number of results to retrieve
distances, indices= index.search(query_embedding,k)

#retrieve the top chunks/documents

retrieved_documents = [character_split_texts[i] for i in indices[0]]

#display the retrieved documents/chunks
for doc in retrieved_documents:
    print(doc)
    print("-" * 80)

Revenue, classified by significant product and service offerings, was as follows:
--------------------------------------------------------------------------------
.  Revenue Recognition – Refer to Note 1 to the financial statements  Critical Audit Matter Description  The Company recognizes revenue upon transfer of control of promised products or services to customers in an amount that reflects the consideration the Company expects to receive in exchange for those products or services. The Company offers customers the ability to acquire multiple licenses of software products and services, including cloud-based services, in its customer agreements through its volume licensing programs.
--------------------------------------------------------------------------------
Segment revenue and operating income were as follows during the periods presented:    
  No sales to an individual customer or country other than the United States accounted for more than 10% of revenue for fiscal years 2023, 

In [6]:
#Re-Ranking using a Cross-Encoder

cross_encoder= CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")
pairs=[[query_text, doc] for doc in retrieved_documents]
scores=cross_encoder.predict(pairs)

#Sort the documents by their scores
sorted_indices=np.argsort(scores)[::-1]
top_documents=[retrieved_documents[i] for i in sorted_indices]

#display the re-ranked documents
for i, doc in enumerate(top_documents):
    print(f"Rank {i+1}:")
    print(doc)
    print("-" * 80)


C:\Users\suman\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\suman\.cache\huggingface\hub\models--cross-encoder--ms-marco-MiniLM-L-6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Rank 1:
Revenue Recognition  Revenue is recognized upon transfer of control of promised products or services to customers in an amount that reflects the consideration we expect to receive in exchange for those products or services. We enter into contracts that can include various combinations of products and services, which are generally capable of being distinct and accounted for as separate performance obligations. Revenue is recognized net of allowances for returns and any taxes collected from customers, which are subsequently remitted to governmental authorities.  Nature of Products and Services  Licenses for on-premises software provide the customer with a right to use the software as it exists when made available to the customer. Customers may purchase perpetual licenses or subscribe to licenses, which provide customers with the same functionality and differ mainly in the duration over which the customer benefits from the software
-------------------------------------------------

In [7]:
#Generate Response

#Contactenate the top documents into a single context
context="\n\n".join(top_documents[:5])

#write a generate response function, Pass the query and context to a model. 
#Prepare the 